In [1]:
import cv2
import torch
from super_gradients.training import models
import numpy as np
import math
from numpy import random
from IPython.display import HTML
from base64 import b64encode
import torchvision.models as torch_models
import os
from IPython.display import Video
import gdown
from deep_sort_pytorch.utils.parser import get_config
from deep_sort_pytorch.deep_sort import DeepSort
from torchvision import transforms
from PIL import Image
from torch import nn
import logging
logging.getLogger("super_gradients").setLevel(logging.WARNING)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps") if torch.backends.mps.is_available() else device
print(device)
!nvidia-smi

The console stream is logged into /home/zw2688/sg_logs/console.log


[2023-11-06 14:35:29] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-11-06 14:35:30] WARNING - __init__.py - Failed to import pytorch_quantization
2023-11-06 14:35:35.180453: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 14:35:36.931224: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 14:35:36.931247: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 14:35:36.931278: E tensorflow/compiler/

cuda
Mon Nov  6 14:35:44 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                On  | 00000000:06:00.0 Off |                    0 |
| N/A   29C    P8              25W / 250W |      3MiB / 46080MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+--------------------

In [16]:
def get_video_info(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        raise ValueError("Error: Could not open the video file.")
    
    # Get the frame width and frame height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    fps = cap.get(cv2.CAP_PROP_FPS)

    return cap, fps, frame_width, frame_height

def display_video(video_path):
    return HTML(f"""
        <video alt="test" controls width="640" height="360">
            <source src="{video_path}" type="video/mp4">
        </video>
    """)

def display_compressed_video(input_path):
    # Compressed video path
    compressed_path = "result_compressed.mp4"
    if os.path.exists(compressed_path):
        os.remove(compressed_path)

    os.system(f"ffmpeg -i {input_path} -vcodec libx264 {compressed_path}");

    # Show video
    mp4 = open(compressed_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display_html = """
    <video width=800 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url

    return HTML(display_html)

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

def cls_predict_image(cls_model, img, preprocess = preprocess):
    input_tensor = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        cls_output = cls_model(input_tensor)
    probability = torch.nn.functional.softmax(cls_output[0], dim=0)
    _, predicted_class = torch.max(probability, dim=0)
    
    return predicted_class.item()

def detect_hoop_box(img, cls_model, x1, y1, x2, y2):
    cropped_img = img[y1:y2, x1:x2]
    cropped_img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY))
    # Preprocess the cropped image
    predicted_class = cls_predict_image(cls_model, cropped_img_pil)
    
    return cropped_img_pil, predicted_class




In [11]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video_path = './clips/20230720_Game4.mp4'
cap, fps, frame_width, frame_height = get_video_info(video_path)
classNames = ['basketball', 'basketball-hoops', 'made']
cap = cv2.VideoCapture(video_path)

out=cv2.VideoWriter('output.mp4', fourcc, fps*0.6, (frame_width, frame_height))
score = 0

model = models.get("yolo_nas_s", num_classes=3, checkpoint_path="./yolonas_checkpoints/bball_detect/RUN_20231104_130305_141167/ckpt_best.pth").to(device)

cls_model = torch_models.resnet18(weights=torch_models.ResNet18_Weights.DEFAULT)
num_ftrs = cls_model.fc.in_features
cls_model.fc = nn.Linear(num_ftrs, 2)
cls_model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

state_dict = torch.load("./cls_chkpoint/best_model.pth", map_location=device)#.to(device)
cls_model.load_state_dict(state_dict)
cls_model.to(device)
i = 0
cap.set(cv2.CAP_PROP_POS_MSEC, 3000)
while True:
    ret, frame = cap.read()
    current_time = cap.get(cv2.CAP_PROP_POS_MSEC)
    if ret and current_time < 4000:
        result=list(model.predict(frame, conf=0.5))[0]
        bbox_xyxys=result.prediction.bboxes_xyxy.tolist()
        confidences=result.prediction.confidence
        labels=result.prediction.labels.tolist()
        for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
            bbox=np.array(bbox_xyxy)
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            classname=int(cls)
            class_name=classNames[classname]
            center_x_bbox = (x1 + x2) / 2
            if class_name == "basketball-hoops":
                if x1 > x2 or y1 > y2:
                    continue
                cropped_img, predicted = detect_hoop_box(frame, cls_model, x1, y1, x2, y2)
                if current_time >= 3000 and center_x_bbox < frame_width/2:
                    cropped_img.save(f"cropped/{i}.jpg")
                    i += 1
                if predicted == 1:
                    score += 1
            conf=round(confidence, 2)
            label=f'{class_name}{conf}'

            cv2.rectangle(frame, (x1, y1), (x2, y2), color=(0, 0, 255, 0.6),thickness=2, lineType=cv2.LINE_AA)
            #Create a rectangle above the detected object and add label and confidence score
            t_size=cv2.getTextSize(str(label), 0, fontScale=1/2, thickness=1)[0]
            c2=x1+t_size[0], y1-t_size[1]-3
            cv2.rectangle(frame, (x1, y1), c2, color=(0, 0, 255, 0.6), thickness=-1, lineType=cv2.LINE_AA)
            cv2.putText(frame, str(label), (x1, y1-2), 0, 1/2, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        #cv2.imshow("Frame", frame)
        out.write( frame)
        
    else:
        break


out.release()
cap.release()

In [9]:
display_compressed_video('output.mp4')

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

frame=   30 fps=0.0 q=-1.0 Lsize=     262kB time=00:00:01.50 bitrate=1430.4kbits/s speed=3.16x    
video:261kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.457314%
[libx264 @ 0x55b893cf6d40] frame I:1     Avg QP:19.65  size: 76747
[libx264 @ 0x55b893cf6d40] frame P:10    Avg QP:21.44  size: 14260
[libx264 @ 0x55b893cf6d40] frame B:19    Avg QP:25.69  size:  2471
[libx264 @ 0x55b893cf6d40] consecutive B-frames: 10.0% 13.3% 10.0% 66.7%
[libx264 @ 0x55b893cf6d40] mb I  I16..4: 20.0% 76.0%  4.0%
[libx264 @ 0x55b893cf6d40] mb P  I16..4:  0.9%  4.7%  0.8%  P16..4: 22.7%  6.9%  6.0%  0.0%  0.0%    skip:58.2%
[libx264 @ 0x55b893cf6d40] mb B  I16..4:  0.1%  0.3%  0.1%  B16..8: 18.7%  1.6%  0.6%  direct: 0.4%  skip:78.1%  L0:49.0% L1:46.3% BI: 4.7%
[libx264 @ 0x55b893cf6d40] 8x8 transform intra:74.5% inter:77.2%
[libx264 @ 0x55b893cf6d40] coded y,uvDC,uvAC intra: 67.6% 62.7% 20.4% inter: 6.9% 6.9% 1.7%
[libx264 @ 0x55b893cf6d40] i16 v,h,dc,p: 30% 50% 15%  5%
[li

In [18]:
temp_model = torch.load("./cls_model.pth", map_location=device)
img =  Image.open("./cropped/18.jpg")
img = img.convert("RGB")
cls_predict_image(temp_model,img )
img

In [6]:
def initialize_deepsort():
    # Create the Deep SORT configuration object and load settings from the YAML file
    cfg_deep = get_config()
    cfg_deep.merge_from_file("deep_sort_pytorch/configs/deep_sort.yaml")

    # Initialize the DeepSort tracker
    deepsort = DeepSort(cfg_deep.DEEPSORT.REID_CKPT,
                        max_dist=cfg_deep.DEEPSORT.MAX_DIST,
                        # min_confidence  parameter sets the minimum tracking confidence required for an object detection to be considered in the tracking process
                        min_confidence=cfg_deep.DEEPSORT.MIN_CONFIDENCE,
                        #nms_max_overlap specifies the maximum allowed overlap between bounding boxes during non-maximum suppression (NMS)
                        nms_max_overlap=cfg_deep.DEEPSORT.NMS_MAX_OVERLAP,
                        #max_iou_distance parameter defines the maximum intersection-over-union (IoU) distance between object detections
                        max_iou_distance=cfg_deep.DEEPSORT.MAX_IOU_DISTANCE,
                        # Max_age: If an object's tracking ID is lost (i.e., the object is no longer detected), this parameter determines how many frames the tracker should wait before assigning a new id
                        max_age=cfg_deep.DEEPSORT.MAX_AGE, n_init=cfg_deep.DEEPSORT.N_INIT,
                        #nn_budget: It sets the budget for the nearest-neighbor search.
                        nn_budget=cfg_deep.DEEPSORT.NN_BUDGET,
                        use_cuda=True
        )

    return deepsort

deepsort = initialize_deepsort()

[2023-11-02 15:03:02] INFO - feature_extractor.py - Loading weights from deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!


In [7]:
names = classNames
colors = [[random.randint(0, 255) for _ in range(3)]
          for _ in range(len(names))]
palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

In [19]:
def compute_color_for_labels(class_name):
    """
    Function that adds fixed color depending on the class
    """
    if class_name == 'person':
        color = (0, 0, 255)  # Red color for 'person'
    elif class_name == 'sports ball':
        color = (0, 255, 0)  # Green color for 'sports ball'
    else:
        color = (0, 0, 255)  
        
    return tuple(color) 

def draw_boxes(frame, bbox, identities=None, categories=None, names=None, offset=(0,0)):
    for i, box in enumerate(bbox):
        x1, y1,  x2, y2 = [int(i) for i in box]
        x1 += offset[0]
        x2 += offset[0]
        y1 += offset[0]
        y2 += offset[0]
        cat = int(categories[i]) if categories is not None else 0
        cat = classNames[cat]
        id = int(identities[i]) if identities is not None else 0
        #Create Bounding Boxes around the Detected Objects
        cv2.rectangle(frame, (x1, y1), (x2, y2), color= compute_color_for_labels(cat),thickness=2, lineType=cv2.LINE_AA)
        label = str(id) + ":" + cat
        (w,h), _ = cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1/2, thickness=1)
        #Create a rectangle above the detected object and add label and confidence score
        t_size=cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1/2, thickness=1)[0]
        c2=x1+t_size[0], y1-t_size[1]-3
        cv2.rectangle(frame, (x1, y1), c2, color=compute_color_for_labels(cat), thickness=-1, lineType=cv2.LINE_AA)
        cv2.putText(frame, str(label), (x1, y1-2), 0, 1/2, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    return frame

In [20]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
output=cv2.VideoWriter('tracking_out.mp4', fourcc, fps*0.6, (frame_width, frame_height))
while True:
    xywh_bboxs = []
    confs = []
    oids = []
    outputs = []
    ret, frame = cap.read()
    if ret:
        result = list(model.predict(frame, conf=0.5))[0]
        bbox_xyxys = result.prediction.bboxes_xyxy.tolist()
        confidences = result.prediction.confidence
        labels = result.prediction.labels.tolist()
        for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
            bbox = np.array(bbox_xyxy)
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            conf = math.ceil((confidence*100))/100
            cx, cy = int((x1+x2)/2), int((y1+y2)/2)
            bbox_width = abs(x1-x2)
            bbox_height = abs(y1-y2)
            xcycwh = [cx, cy, bbox_width, bbox_height]
            xywh_bboxs.append(xcycwh)
            confs.append(conf)
            oids.append(int(cls))
        xywhs = torch.tensor(xywh_bboxs)
        confss= torch.tensor(confs)
        outputs = deepsort.update(xywhs, confss, oids, frame)
        if len(outputs)>0:
            bbox_xyxy = outputs[:,:4]
            identities = outputs[:, -2]
            object_id = outputs[:, -1]
            draw_boxes(frame, bbox_xyxy, identities, object_id)
        output.write(frame)
    else:
        break

output.release()
cap.release()

In [4]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
output=cv2.VideoWriter('tracking_out.mp4', fourcc, fps*0.6, (frame_width, frame_height))
while True:
    xywh_bboxs = []
    confs = []
    oids = []
    outputs = []
    ret, frame = cap.read()
    if ret:
        result = list(model(frame, 
                        stream = True, 
                        device = device, 
                        conf = 0.4))[0]
        bbox_xyxys = result.prediction.bboxes_xyxy.tolist()
        confidences = result.prediction.confidence
        labels = result.prediction.labels.tolist()
        for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
            bbox = np.array(bbox_xyxy)
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            conf = math.ceil((confidence*100))/100
            cx, cy = int((x1+x2)/2), int((y1+y2)/2)
            bbox_width = abs(x1-x2)
            bbox_height = abs(y1-y2)
            xcycwh = [cx, cy, bbox_width, bbox_height]
            xywh_bboxs.append(xcycwh)
            confs.append(conf)
            oids.append(int(cls))
        xywhs = torch.tensor(xywh_bboxs)
        confss= torch.tensor(confs)
        outputs = deepsort.update(xywhs, confss, oids, frame)
        if len(outputs)>0:
            bbox_xyxy = outputs[:,:4]
            identities = outputs[:, -2]
            object_id = outputs[:, -1]
            draw_boxes(frame, bbox_xyxy, identities, object_id)
        output.write(frame)
    else:
        break

output.release()
cap.release()

NameError: name 'fps' is not defined

In [21]:
display_video("tracking_out.mp4")